In [ ]:
import eradiate
import matplotlib.pyplot as plt
import mitsuba as mi
import numpy as np
import pymap3d as pm
import xarray as xr
from eradiate import KernelContext, fresolver
from eradiate import unit_registry as ureg
from eradiate.kernel import (
    KernelSceneParameterFlags,
    SceneParameter,
    SearchSceneParameter,
)
from eradiate.scenes.spectra import InterpolatedSpectrum

eradiate.set_mode("mono")
fresolver.append("../data")

In [ ]:
r = eradiate.constants.EARTH_RADIUS.m_as("m")
ell = pm.ellipsoid.Ellipsoid(r, r, model="sphere")
c = [2.23, 31.02, 530.0]  # lon, lat, alt
target_center = pm.geodetic2enu(c[1], c[0], c[2], c[1], c[0], -r, ell) * ureg.m

target_center_north = np.array([0, 1, 0])
target_center_east = np.array([1, 0, 0])
target_center_up = np.array([0, 0, 1])

In [ ]:
reflectance = xr.load_dataset(fresolver.resolve("bsdf/rpv_sand.nc"))["rho_0"]
sand_spectrum = s = InterpolatedSpectrum.from_dataarray(dataarray=reflectance)


def sand_reflectance(ctx: KernelContext) -> float:
    return s.eval(ctx.si).m_as("dimensionless")


sand_kpmap = {
    "_mat_sand.reflectance.value": SceneParameter(
        sand_reflectance,
        search=SearchSceneParameter(
            node_type=mi.BSDF,
            node_id="_mat_sand",
            parameter_relpath="reflectance.value",
        ),
        flags=KernelSceneParameterFlags.SPECTRAL,
    )
}

In [ ]:
kdict = {
    "_mat_sand": {
        "type": "diffuse",
        "id": "_mat_sand",
    },
    "_shape_algeria-5": {
        "type": "ply",
        "filename": str(fresolver.resolve("ply/algeria-5.ply")),
        "face_normals": True,
        "bsdf": {"type": "ref", "id": "_mat_sand"},
    },
    "_shape_background": {
        "type": "sphere",
        "radius": r,
        "bsdf": {"type": "ref", "id": "_mat_sand"},
    },
}
kpmap = {**sand_kpmap}

illumination = {"type": "directional", "zenith": 30.0, "azimuth": 200.0}

t = target_center.m
o = t + target_center_east * 1000.0 + target_center_up * 50.0
u = target_center_up
measure = {
    "type": "perspective",
    # "film_resolution": (160, 80),  # Fast preview
    "film_resolution": (750, 375),  # Medium resolution
    # "film_resolution": (1500, 750),  # High resolution
    "far_clip": 1e20,  # Set it to a very large value to prevent any clipping
    "target": t,
    "origin": o,
    "up": u,
    "srf": {"type": "delta", "wavelengths": [440, 550, 660]},
}

exp = eradiate.experiments.AtmosphereExperiment(
    geometry="spherical_shell",
    surface=None,
    atmosphere={
        "type": "heterogeneous",
        "molecular_atmosphere": {},
        "particle_layers": {
            "type": "particle_layer",
            "bottom": 0.0,
            "top": 10e3,
            "distribution": {"type": "exponential"},
            "tau_ref": 0.1,
            "dataset": "govaerts_2021-desert",
        },
    },
    # atmosphere=None,
    illumination=illumination,
    measures=measure,
    kdict=kdict,
    kpmap=kpmap,
)
exp.init()

In [ ]:
spp = 16  # Quick preview
# spp = 1024  # Nice render with low noise
result = eradiate.run(exp, spp=spp)

In [ ]:
img = (
    eradiate.xarray.interp.dataarray_to_rgb(
        result.squeeze()["radiance"],
        channels=[("w", 660), ("w", 550), ("w", 440)],
        gamma_correction=True,
    )
    * 1.8  # Adjust brightness with this multiplier
)
plt.imshow(img)
plt.axis("off")